In [12]:
import requests
import numpy as np
import pandas as pd

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from futures_sign import send_signed_request, send_public_request
from key import KEY, SECRET

In [13]:
symbol='ETHUSDT'
client = Client(KEY,SECRET)

In [14]:
# Get last 500 kandels 5 minutes for Symbol

def get_futures_klines(symbol,limit=500):
    x = requests.get('https://www.binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(limit)+'&interval=5m')
    df=pd.DataFrame(x.json())
    df.columns=['open_time','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['open']=df['open'].astype(float)
    df['high']=df['high'].astype(float)
    df['low']=df['low'].astype(float)
    df['close']=df['close'].astype(float)
    df['volume']=df['volume'].astype(float)
    return(df)

In [15]:
# Open position for Sybol with 

def open_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        

                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
       
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
           ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)

In [16]:
# Close position for symbol with quantity

def close_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)
    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
                
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)

In [17]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[positions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol']==symbol]['leverage'])
    entryprice = positions[positions['symbol']==symbol]['entryPrice']
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totalWalletBalance']),2)
    if a>0:
        pos = "long"
    elif a<0:
        pos = "short"
    else: 
        pos = ""
    return([pos,a,profit,leverage,balance,round(float(entryprice),3),0])

In [18]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [19]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [20]:
open_position(symbol,'long',0.01)

In [25]:
get_opened_positions(symbol)

['', 0.0, 0.0, 5, 13.76, 0.0, 0]

In [22]:
close_position(symbol,'long',0.02)

In [23]:
get_opened_positions(symbol)

['', 0.0, 0.11951199, 5, 11.83, 0.0, 0]

In [24]:
get_futures_klines(symbol)

,open_time,open,high,low,close,volume,close_time
0,1685329800000,1908.26,1908.66,1906.36,1907.32,9846.311,1685330099999
1,1685330100000,1907.33,1907.93,1906.34,1906.87,6881.344,1685330399999
2,1685330400000,1906.86,1907.68,1905.80,1906.65,7430.771,1685330699999
3,1685330700000,1906.65,1908.72,1906.39,1907.87,7763.731,1685330999999
4,1685331000000,1907.88,1908.20,1904.76,1905.64,15089.906,1685331299999
...,...,...,...,...,...,...,...
495,1685478300000,1905.27,1905.44,1904.33,1904.84,5172.380,1685478599999
496,1685478600000,1904.84,1905.98,1904.33,1905.44,2653.922,1685478899999
497,1685478900000,1905.45,1905.64,1904.22,1904.65,1452.386,1685479199999
498,1685479200000,1904.65,1905.33,1904.65,1905.32,907.513,1685479499999


In [21]:
close_position(symbol,'long',0.01)